In [22]:
#HW 2
#1.Given 1000 records in a dataset, 1000 models are trained with 999 records as part of
#the training sample and the remaining 1 sample for testing, and the error rate is averaged out,
#this validation technique is called:
    #c. LOOCV

#2.In k-fold cross validation technique, the value of k being small could lead to which of
#the following in relation to the error rate
    #c. High Bias and Low variance 
    
#3.In k-fold cross validation technique, the value of k being large could lead to which of
#the following in relation to the error rate
    #b. Low bias and high variation
    
#4.Explain what regularization is and why it is useful
    #Regularization is the process finding ways to ajust ML models 
    #to reduce losses and prevent overfitting/underfitting. 

In [1]:
import boto3
import pandas as pd; pd.set_option('display.max_column', 100)
import numpy as np

from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.preprocessing import MinMaxScaler

s3= boto3.resource('s3')
bucket_name= 'morgangant-bata-445-bucket'
bucket= s3.Bucket(bucket_name)

file_key= 'framingham.csv'

bucket_object= bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

#reading the datefile
heart = pd.read_csv(file_content_stream)
heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [2]:
#Removing na values
heart= heart.dropna()

In [13]:
#Defining input and target variables
x= heart.drop(columns= ['TenYearCHD'], axis=1)
y= heart['TenYearCHD']

#Defining the lists to store results
f1score_1= list()
f1score_2= list()

#Defining the folds
kf=KFold(n_splits= 5, shuffle= True)


for train_idx, val_idx in kf.split(x):
    #Splitting data
    x_train, x_val= x.iloc[train_idx], x.iloc[val_idx]
    y_train, y_val= y.iloc[train_idx], y.iloc[val_idx]
    
    #Model 1 
    x1= x_train[['age', 'currentSmoker', 'totChol', 'sysBP', 'diaBP', 'BMI', 'heartRate', 'glucose']] 
    x_val_1= x_val[['age', 'currentSmoker', 'totChol', 'sysBP', 'diaBP', 'BMI', 'heartRate', 'glucose']] 
    
    #Scaling
    scaler=MinMaxScaler()
    x1=scaler.fit_transform(x1)
    x_val_1=scaler.fit_transform(x_val_1)
   
    
    #Building the linear model
    md1= LogisticRegression().fit(x1, y_train)
    
    #Predicting on the validation set
    md1_pred= md1.predict_proba(x_val_1)[:, 1]
    
    #Setting 25% threshold
    md1_labels=np.where(md1_pred<0.25,0,1)
    
    #Computing F1 score and storing it
    md1_f1= f1_score(y_val, md1_labels)
    f1score_1.append(md1_f1)
    
    
    #Model 2
    x2= x_train[['age', 'currentSmoker', 'totChol', 'BMI', 'heartRate', 'glucose']]
    x_val_2= x_val[['age', 'currentSmoker', 'totChol', 'BMI', 'heartRate', 'glucose']] 
    
    #Scaling
    scaler=MinMaxScaler()
    x2=scaler.fit_transform(x2)
    x_val_2=scaler.fit_transform(x_val_2)
    
    #Building the linear model
    md2= LogisticRegression().fit(x2, y_train)
    
    #Predicting on the validation set
    md2_pred= md2.predict_proba(x_val_2)[:, 1]
     
    #Setting 25% threshold
    md2_labels=np.where(md2_pred<0.25,0,1)
    
    #Computing F1 score and storing it
    md2_f1= f1_score(y_val, md2_labels)
    f1score_2.append(md2_f1)

In [16]:
print("The average F1-Score of model 1 is:", np.mean(f1score_1))
print("The average F1-Score of model 2 is:", np.mean(f1score_2))

The average F1-Score of model 1 is: 0.3580344663748408
The average F1-Score of model 2 is: 0.3521192581765447


In [17]:
#Based on the F1 scores from both models, Model 1 has higher average F1 values so we would want to continue with Model 1 over Model 2!!